In [61]:
#train
import read_player_stats
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn import metrics
pd.options.display.max_columns = 1000
import matplotlib.pyplot as plt
%matplotlib inline

In [62]:
def make_total_data(seasons=range(2004,2014),pages=[0,1]):
	total_df = None
	for season in seasons:
		for page in pages:
			df = read_player_stats.rb_stats(season,page)
			if total_df is None:
				total_df = df
			else:
				total_df = total_df.append(df, ignore_index = True)
			
	total_df.sort_index(by=['Name','Season'], inplace=True)
	return total_df

def merge_seasons(df_season1, df_season2):
	df_dropped1 = df_season1.drop(['Season', 'Team'], axis=1)
	df_dropped2 = df_season2.drop(['Season', 'Team'], axis=1)
	
	merged = pd.merge(df_dropped1,df_dropped2, on='Name', how='outer', suffixes=('_1', '_2'))
	return merged

def make_training_df(total_df, seasons=range(2004,2013)):
	training_data_df = None
	for season in seasons[:-1]:
		df1 = total_df[total_df.Season == season]
		df2 = total_df[total_df.Season == season+1]
		label_df = total_df[total_df.Season == season+2]
		labeled = pd.merge(merge_seasons(df1, df2), label_df[['Name','FFP']], on='Name')
		if training_data_df is None:
			training_data_df = labeled
		else:
			training_data_df = training_data_df.append(labeled, ignore_index=True)
		
	training_data_nadropped = training_data_df.dropna() #dont train on missing seasons
	return training_data_nadropped

In [63]:
total_df = make_total_data(seasons=range(2004,2015), pages=[0,1])
train_df = make_training_df(total_df, seasons=range(2004,2012))

In [64]:
X_train = np.array(train_df.drop(['Name','FFP'], axis=1))
y_train = np.array(train_df['FFP'])

In [65]:
model = RandomForestRegressor(n_estimators=500)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [66]:
eval_df = make_training_df(total_df, seasons=range(2012,2015))
X_eval = np.array(eval_df.drop(['Name','FFP'], axis=1))

In [67]:
y_eval = model.predict(X_eval)

In [68]:
pred_summary = eval_df[['Name']]
pred_summary['Predicted'] = y_eval
actual = total_df[total_df.Season == 2014]
actual = actual[['Name','FFP']]
pred_summary = pd.merge(pred_summary, actual, on='Name', how='inner')

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [69]:
pred_summary

,Name,Predicted,FFP
0,AhmadBradshaw,108.886217,120.5
1,AlfredMorris,134.230000,170.9
2,AnthonyDixon,93.985060,60.1
3,ArianFoster,106.434400,235.3
4,BenTate,98.272200,67.0
5,BernardPierce,99.601477,49.6
6,BilalPowell,111.080400,29.3
7,BryceBrown,82.415483,30.2
8,CJSpiller,105.825000,48.4
9,ChrisIvory,115.455800,136.3
